In [1]:
import os
import numpy as np
import imagecodecs
from skimage.io import imsave, imread

tr_path = './tr/'
ts_path = './ts/'
pred_path = './preds/'

image_rows = 420
image_cols = 580


In [2]:
def create_train_data():
    tr_data_path = os.path.join(tr_path)
    images = os.listdir(tr_data_path)
    total = int(len(images) / 2)

    imgs = np.ndarray((total, image_rows, image_cols), dtype=np.uint8)
    imgs_mask = np.ndarray((total, image_rows, image_cols), dtype=np.uint8)

    i = 0
    print('------------------------------')
    print('Fetching training images...')
    print('------------------------------')
    for image_name in images:
        if 'mask' in image_name:
            continue
        image_mask_name = image_name.split('.')[0] + '_mask.tif'
        img = imread(os.path.join(tr_data_path, image_name), as_gray=True)
        img_mask = imread(os.path.join(tr_data_path, image_mask_name), as_gray=True)

        img = np.array([img])
        img_mask = np.array([img_mask])

        imgs[i] = img
        imgs_mask[i] = img_mask

        print("Converted ", (i+1), "/", total, " images")
        i += 1
    print('Loading done.')

    np.save('imgs_train.npy', imgs)
    np.save('imgs_mask_train.npy', imgs_mask)
    print('Saving to .npy files done.')

In [3]:
def create_test_data():
    ts_data_path = os.path.join(ts_path)
    images = os.listdir(ts_data_path)
    total = len(images)

    imgs = np.ndarray((total, image_rows, image_cols), dtype=np.uint8)
    imgs_id = np.ndarray((total, ), dtype=np.int32)

    i = 0
    print('------------------------------')
    print('Creating test images...')
    print('------------------------------')
    for image_name in images:
        if 'mask' in image_name:
            continue
        img_id = int(image_name.split('.')[0])
        img = imread(os.path.join(ts_data_path, image_name), as_gray=True)

        img = np.array([img])

        imgs[i] = img
        imgs_id[i] = img_id

        print("Converted ", (i+1), "/", total, " images")
        i += 1
    print('Loading done.')

    np.save('imgs_test.npy', imgs)
    np.save('imgs_id_test.npy', imgs_id)
    print('Saving to .npy files done.')

In [4]:
def load_train_data():
    imgs_train = np.load('imgs_train.npy')
    imgs_mask_train = np.load('imgs_mask_train.npy')
    return imgs_train, imgs_mask_train

In [5]:
def load_test_data():
        imgs_test = np.load('imgs_test.npy')
        imgs_id = np.load('imgs_id_test.npy')
        return imgs_test, imgs_id

In [6]:
create_train_data()
create_test_data()

------------------------------
Fetching training images...
------------------------------
Converted  1 / 240  images
Converted  2 / 240  images
Converted  3 / 240  images
Converted  4 / 240  images
Converted  5 / 240  images
Converted  6 / 240  images
Converted  7 / 240  images
Converted  8 / 240  images
Converted  9 / 240  images
Converted  10 / 240  images
Converted  11 / 240  images
Converted  12 / 240  images
Converted  13 / 240  images
Converted  14 / 240  images
Converted  15 / 240  images
Converted  16 / 240  images
Converted  17 / 240  images
Converted  18 / 240  images
Converted  19 / 240  images
Converted  20 / 240  images
Converted  21 / 240  images
Converted  22 / 240  images
Converted  23 / 240  images
Converted  24 / 240  images
Converted  25 / 240  images
Converted  26 / 240  images
Converted  27 / 240  images
Converted  28 / 240  images
Converted  29 / 240  images
Converted  30 / 240  images
Converted  31 / 240  images
Converted  32 / 240  images
Converted  33 / 240  im

In [7]:
from __future__ import print_function

import os
from skimage.transform import resize
from skimage.io import imsave
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras import backend as K




def dice_coef(y_true, y_pred):
    y_true_flattened = K.flatten(y_true)
    y_pred_flattened = K.flatten(y_pred)
    intersection = K.sum(y_true_flattened * y_pred_flattened)
    return (2. * intersection + smooth) / (K.sum(y_true_flattened) + K.sum(y_pred_flattened) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

In [8]:
K.set_image_data_format('channels_last')  # TF dimension ordering in this code

img_rows = 96
img_cols = 96
smooth = 1.


def unet():
    inputs = Input((img_rows, img_cols, 1))
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])
    model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=[dice_coef])
    model.summary()
    return model

In [9]:
def preprocess(imgs):
    imgs_p = np.ndarray((imgs.shape[0], img_rows, img_cols), dtype=np.uint8)
    for i in range(imgs.shape[0]):
        imgs_p[i] = resize(imgs[i], (img_cols, img_rows), preserve_range=True)

    imgs_p = imgs_p[..., np.newaxis]
    return imgs_p

In [10]:
def train_and_predict():
    print('------------------------------')
    print('Loading and preprocessing training data...')
    print('------------------------------')
    imgs_train, imgs_mask_train = load_train_data()

    imgs_train = preprocess(imgs_train)
    imgs_mask_train = preprocess(imgs_mask_train)

    imgs_train = imgs_train.astype('float32')
    mean = np.mean(imgs_train)  
    std = np.std(imgs_train)  

    imgs_train -= mean
    imgs_train /= std

    imgs_mask_train = imgs_mask_train.astype('float32')
    imgs_mask_train /= 255.  # scale masks to [0, 1]

    print('------------------------------')
    print('Creating and compiling model...')
    print('------------------------------')
    model = unet()
    model_checkpoint = ModelCheckpoint('weights.h5', monitor='val_loss', save_best_only=True)

    print('------------------------------')
    print('Fitting model...')
    print('------------------------------')
    model.fit(imgs_train, imgs_mask_train, batch_size=16, epochs=50, verbose=1, shuffle=True,
              callbacks=[model_checkpoint])

    print('------------------------------')
    print('Loading and preprocessing test data...')
    print('------------------------------')
    imgs_test, imgs_id_test = load_test_data()
    imgs_test = preprocess(imgs_test)

    imgs_test = imgs_test.astype('float32')
    imgs_test -= mean
    imgs_test /= std

    print('------------------------------')
    print('Loading saved weights...')
    print('------------------------------')
    model.save_weights('weights.h5')

    print('------------------------------')
    print('Predicting masks on test data...')
    print('------------------------------')
    imgs_mask_test = model.predict(imgs_test, verbose=1)
    np.save('imgs_mask_test.npy', imgs_mask_test)

    print('------------------------------')
    print('Saving predicted masks to files...')
    print('------------------------------')

    if not os.path.exists(pred_path):
        os.mkdir(pred_path)
    for image, image_id in zip(imgs_mask_test, imgs_id_test):
        image = (image[:, :, 0] * 255.).astype(np.uint8)
        imsave(os.path.join(pred_path, str(image_id) + '_pred.png'), image)

In [11]:
train_and_predict()

------------------------------
Loading and preprocessing training data...
------------------------------
------------------------------
Creating and compiling model...
------------------------------
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 96, 96, 1)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 96, 96, 32)   320         ['input_1[0][0]']                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 96, 96, 32)   9248        ['conv2d[0][0]']                 
                                                                                             

C:\Users\Harshal\AppData\Roaming\Python\Python311\site-packages\keras\optimizers\legacy\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


15/15 [==============================] - 20s 1s/step - loss: -0.0142 - dice_coef: 0.0142
Epoch 2/50
15/15 [==============================] - 18s 1s/step - loss: -0.0143 - dice_coef: 0.0143
Epoch 3/50
15/15 [==============================] - 18s 1s/step - loss: -0.0144 - dice_coef: 0.0144
Epoch 4/50
15/15 [==============================] - 18s 1s/step - loss: -0.0145 - dice_coef: 0.0145
Epoch 5/50
15/15 [==============================] - 18s 1s/step - loss: -0.0146 - dice_coef: 0.0146
Epoch 6/50
15/15 [==============================] - 18s 1s/step - loss: -0.0147 - dice_coef: 0.0147
Epoch 7/50
15/15 [==============================] - 18s 1s/step - loss: -0.0149 - dice_coef: 0.0149
Epoch 8/50
15/15 [==============================] - 18s 1s/step - loss: -0.0150 - dice_coef: 0.0150
Epoch 9/50
15/15 [==============================] - 18s 1s/step - loss: -0.0152 - dice_coef: 0.0152
Epoch 10/50
15/15 [==============================] - 18s 1s/step - loss: -0.0155 - dice_coef: 0.0155
Epoch 11/5

C:\Users\Harshal\AppData\Local\Temp\ipykernel_9996\1950911102.py:61: UserWarning: ./preds/31_pred.png is a low contrast image
  imsave(os.path.join(pred_path, str(image_id) + '_pred.png'), image)
C:\Users\Harshal\AppData\Local\Temp\ipykernel_9996\1950911102.py:61: UserWarning: ./preds/310_pred.png is a low contrast image
  imsave(os.path.join(pred_path, str(image_id) + '_pred.png'), image)
C:\Users\Harshal\AppData\Local\Temp\ipykernel_9996\1950911102.py:61: UserWarning: ./preds/3100_pred.png is a low contrast image
  imsave(os.path.join(pred_path, str(image_id) + '_pred.png'), image)
C:\Users\Harshal\AppData\Local\Temp\ipykernel_9996\1950911102.py:61: UserWarning: ./preds/3101_pred.png is a low contrast image
  imsave(os.path.join(pred_path, str(image_id) + '_pred.png'), image)
C:\Users\Harshal\AppData\Local\Temp\ipykernel_9996\1950911102.py:61: UserWarning: ./preds/3103_pred.png is a low contrast image
  imsave(os.path.join(pred_path, str(image_id) + '_pred.png'), image)
C:\Users\Hars